# 本番

In [4]:
import sys
import json
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text

# パス設定
sys.path.append("../configs")
sys.path.append('../db/')

import config as cf
from config import seed_everything

import src.db.models as models
import src.db.schemas as schemas
import src.db.cruds as cruds

# パス読み込み
RAW_MODELS_PATH = cf.RAW_MODELS_PATH[0]
PREPROCESSED_MODELS_PATH = cf.PREPROCESSED_MODELS_PATH[0]
QUANTIZED_MODELS_PATH = cf.QUANTIZED_MODELS_PATH[0]
MODELS_DB_PATH= cf.MODELS_DB_PATH[0]

# DB設定読み込み
with open('C:/Users/thyt/confidential_files/Postgresql/config.json', 'r', encoding='utf-8') as file:
    config = json.load(file)

class DBConfigurations:
    postgres_username = config["POSTGRES_USER"]
    postgres_password = config["POSTGRES_PASSWORD"]
    postgres_port = int(config["POSTGRES_PORT"])
    postgres_db = config["POSTGRES_DB"]
    postgres_server = config["POSTGRES_SERVER"]
    sql_alchemy_database_url = f"postgresql://{postgres_username}:{postgres_password}@{postgres_server}:{postgres_port}/{postgres_db}"

engine = create_engine(DBConfigurations.sql_alchemy_database_url, pool_recycle=3600, echo=False)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
db = SessionLocal()


## DB操作

In [33]:
### データベースのテーブルの確認
from sqlalchemy import inspect

inspector = inspect(engine)
table_names = inspector.get_table_names()

tmp = {}
# 各テーブルのスキーマ情報を表示
for table_name in table_names:
    print(f"テーブル名: {table_name}")
    
    # テーブルのカラム情報を取得
    columns = inspector.get_columns(table_name)
    
    # 各カラムの情報を表示
    for column in columns:
        print(f"  {column['name']}, 型: {column['type']}")

テーブル名: experiments
  experiment_id, 型: VARCHAR(255)
  model_id, 型: VARCHAR(255)
  model_version_id, 型: VARCHAR(255)
  parameters, 型: JSON
  training_dataset_path, 型: TEXT
  validation_dataset_path, 型: TEXT
  test_dataset_path, 型: TEXT
  evaluations, 型: JSON
  artifact_file_paths, 型: JSON
  created_datetime, 型: TIMESTAMP
テーブル名: models
  model_id, 型: VARCHAR(255)
  parent_model_id, 型: VARCHAR(255)
  project_id, 型: VARCHAR(255)
  model_name, 型: VARCHAR(255)
  description, 型: TEXT
  created_datetime, 型: TIMESTAMP
テーブル名: projects
  project_id, 型: VARCHAR(255)
  project_name, 型: VARCHAR(255)
  description, 型: TEXT
  created_datetime, 型: TIMESTAMP


In [42]:
sql_query = text("SELECT * FROM projects")
results = db.execute(sql_query)
data = [result for result in results]
df_projects = pd.DataFrame(data)

sql_query = text("SELECT * FROM models")
results = db.execute(sql_query)
data = [result for result in results]
df_models = pd.DataFrame(data)

sql_query = text("SELECT * FROM experiments")
results = db.execute(sql_query)
data = [result for result in results]
df_experiments = pd.DataFrame(data)
# df_experiments

In [43]:
df_models

,model_id,parent_model_id,project_id,model_name,description,created_datetime
0,d1ce88,None,207503,AAAAAA_20231125_011819_model.onnx,RAWモデル,2023-11-25 01:37:24.019569+09:00
1,ad36fa,d1ce88,207503,AAAAAA_20231125_011819_model_infer.onnx,前処理済みのモデル,2023-11-25 01:37:24.027590+09:00
2,d7e4ba,d1ce88,207503,AAAAAA_20231125_011819_model_infer_dq_qop_wa_u...,動的量子化済みモデル,2023-11-25 01:37:24.030803+09:00
3,16003b,d1ce88,207503,AAAAAA_20231125_011819_model_infer_sq_qdq_wa_s...,静的量子化（QDQ）済みモデル,2023-11-25 01:37:24.033545+09:00
4,94f825,d1ce88,207503,AAAAAA_20231125_011819_model_infer_sq_qop_wa_s...,静的量子化（QOperator）済みモデル,2023-11-25 01:37:24.035941+09:00


In [58]:
# df_projectsとdf_modelsをproject_idでleft join
df_joined = pd.merge(df_projects, df_models, on="project_id", how="left")

# 結合後のデータフレームdf_joinedとdf_experimentsをmodel_idでleft join
final_df = pd.merge(df_joined, df_experiments, on="model_id", how="left")

tmpdf = pd.DataFrame()
# evaluations列をイテレーション
for index, row in final_df.iterrows():
    tmpdf = pd.concat([tmpdf,pd.DataFrame([row['evaluations']])],axis=0)

tmpdf = tmpdf.drop(columns=[0])
tmpdf

,モデルの条件,ファイルサイズ/MB,正解率/%,推論時間/s
0,NaN,NaN,NaN,NaN
0,AAAAAA_20231125_011819_model_infer.onnx,44.720849 MB,0.8137,7.883786
0,AAAAAA_20231125_011819_model_infer_dq_qop_wa_u...,11.234564 MB,0.8131,24.908645
0,AAAAAA_20231125_011819_model_infer_sq_qdq_wa_s...,11.236187 MB,0.8133,7.425493
0,AAAAAA_20231125_011819_model_infer_sq_qop_wa_s...,11.217222 MB,0.8132,6.841749


In [97]:
# # SQL クエリの実行

# print("project_id, project_name, description, created_datetime,")
# sql_query = text("SELECT * FROM projects")
# results = db.execute(sql_query)
# for result in results:
#     print(result)
# print("\n")
# print("model_id, parent_model_id, project_id, model_name, description, created_datetime,")
# # SQL クエリの実行
# sql_query = text("SELECT * FROM models")
# results = db.execute(sql_query)
# for result in results:
#     print(result)
# print("\n")
# print("experiment_id, model_id, model_version_id, parameters, training_dataset_path, validation_dataset_path, test_dataset_path, evaluations, artifact_file_paths, created_datetime,")
# # SQL クエリの実行
# sql_query = text("SELECT * FROM experiments")
# results = db.execute(sql_query)
# for result in results:
#     print(result)


### レコード削除

In [40]:
# SQL DELETE文を定義
# parent_model_id = "0706d0"
# sql_query = text(f"DELETE FROM models WHERE parent_model_id = '{parent_model_id}'")
# db.execute(sql_query)
# db.commit()

# SQL DELETE文を定義
model_id = "0706d0"
sql_query = text(f"DELETE FROM models WHERE model_id = '{model_id}'")
db.execute(sql_query)
db.commit()

# SQL DELETE文を定義
# experiment_id = "210c3c"
# sql_query = text(f"DELETE FROM experiments WHERE experiment_id = '{experiment_id}'")
# db.execute(sql_query)
# db.commit()

In [98]:
# SQL DELETE文を定義
# sql_query = text(f"DELETE FROM experiments")
# db.execute(sql_query)
# db.commit()

# SQL DELETE文を定義
# sql_query = text(f"DELETE FROM models")
# db.execute(sql_query)
# db.commit()



In [17]:
db.execute(text("ROLLBACK"))